### Imports

Import necessary libaries and bring in New York Philharmonic JSON data.

In [ ]:
import os
import json
import pandas as pd

# Import New York Phil Performance History from JSON file to a dictionary
perf_history = {}

with open('../nyphilarchive_performance_history/Programs/json/complete.json', 'r') as json_file:
    perf_history = json.load(json_file)

perf_history_records = perf_history['programs']

### Clean Data

This code unnests the JSON performance history, organizing the dataset around "works" instead of "programs."

In the final DataFrame, each row represents the performance of a single work on a particular NY Phil program. The date associated with the performance is the first date a program was performed.

In [8]:
# Function to return the earliest date from an array of concert information
def firstConcertDate(concert_array):
    concert_dates = []

    for concert in concert_array:
        concert_dates.append(concert['Date'])

    concert_dates.sort()
    earliest_date = concert_dates[0]

    return earliest_date

# Convert dictionary from JSON to a DataFrame
perf_df = pd.DataFrame(perf_history_records)

# Create new DataFrame by exploding works column, creating a new row for each work on a program
perf_df_works_exploded = (perf_df
                          .explode('works')
                          .reset_index()
                          )

# Split array of 'works' info into multiple columns and drop redundant fields
works_and_programs = perf_df_works_exploded[['programID', 'works']]
works_dict = works_and_programs.to_dict(orient='records')
works_df = (pd.json_normalize(works_dict, sep='_')
            .drop(['programID', 'works','works_soloists', 'works_conductorName'], axis=1)
            .fillna('')
            )

# Aggregate all 'movement' information into 'works_movement' column
works_df['works_movement__'] = works_df[['works_movement__', 'works_movement_em']].agg(' '.join, axis=1).str.strip()
works_df['works_movement'] = works_df[['works_movement', 'works_movement__']].agg(' '.join, axis=1).str.strip()

# Aggregate all 'workTitle' information into 'works_workTitle' column
works_df['works_workTitle__'] = works_df[['works_workTitle__', 'works_workTitle_em']].agg(' '.join, axis=1).str.strip()
works_df['works_workTitle'] = works_df[['works_workTitle', 'works_workTitle__']].agg(' '.join, axis=1).str.strip()

# Change column names to singular/lowercase
works_df_cleaned = (works_df
                    .rename(columns={
                        'works_ID' : "work_id",
                        "works_composerName" : "work_composer_name",
                        "works_workTitle" : "work_title",
                        "works_movement" : "work_movement"})
                    )

# Concatonate the exploded/cleaned 'performance' and 'works' DataFrames, renaming 'programID' to match the new naming convention
work_history_df = (pd
                   .concat([perf_df_works_exploded, works_df_cleaned], axis=1)
                   .rename(columns={'programID' : 'program_id'})
                   )

# Create a column ('date') that specifies the first date the work was performed on a concert for the associated program
work_history_df['date'] = work_history_df['concerts'].apply(firstConcertDate)

# Remove movement information from work_id
work_history_df['work_id'] = work_history_df['work_id'].apply(lambda x: x.split('*')[0])

# Export to CSV
work_history_df.to_csv('output/ny_phil_unnested_work_history.csv')

### New Works Per Season

This code filters and counts New York Philharmonic's performance history to determine how many "new works" were performed each season.

A "new work" is defined as as piece that has is being performed by the ensemble for the first time in its history.

For comparison, the resulting table also includes the number of "total composers" performed on each season.

In [9]:
# Remove rows with no 'work_id'; remove intermission rows ('work_id' = '0'); set index to be a combination of 'work_id' and 'program_id'; reorder and select relevant columns.
work_history_df_cleaned = (
    work_history_df[(work_history_df['work_id'] != '') & (work_history_df['work_id'] != '0') & (work_history_df['work_id'] != '')]
    .set_index(['work_id', 'program_id'])
    .sort_values(['work_id', 'date'], ascending=[True, True])
    .loc[:,['date', 'season', 'work_title','work_movement','work_composer_name']]
    )

# Filter dataframe for only the first performance of a work
new_work_df = (work_history_df_cleaned
                .groupby('work_id')
                .first()
                .sort_values(by='date'))

# Count the number of new works performed in an orchestral season
new_works_per_season = new_work_df.season.value_counts()
new_works_per_season.sort_index(inplace=True)

#Count of total number of works performed per season
all_works = work_history_df[(work_history_df['work_id'] != '') 
                            & (work_history_df['work_id'] != '0')]
total_works_per_season = (all_works[['season', 'work_id']]
                        .groupby('season')
                        .nunique()
                        .rename(columns={'work_id' : 'total_works'}))

#Merge "new work" and "total work" counts into one dataframe
works_per_season_df = total_works_per_season.merge(new_works_per_season.rename('new_works'), left_index=True, right_index=True)

# Calculate number of "repeat compsers" per season
works_per_season_df['repeat_works'] = (works_per_season_df['total_works']
                                               - works_per_season_df['new_works'])

# Calculate percentage of season with new/repeat composers
works_per_season_df['new_works_p'] = (works_per_season_df['new_works'] 
                                              / works_per_season_df['total_works'])
works_per_season_df['repeat_works_p'] = (works_per_season_df['repeat_works']
                                              / works_per_season_df['total_works'])

# Add season start year for filtering
works_per_season_df['season_year'] = works_per_season_df.index.str[:4].astype(int)

# Export to CSV
works_per_season_df.to_csv('output/ny_phil_new_works_per_season.csv')

### New Composers Per Season

This code filters and counts New York Philharmonic's performance history to determine how many "new composers" were performed  each season.

A "new composer" is counted when the Philharmonic performs a work by a composer whose music has never been performed by the orchestra in its history.

For comparison, the resulting table also includes the number of "total composers" performed on each season.

In [10]:
# Remove rows with no composer history; change index to be a combination of composer name, work id, and program id; reorder and select relevant columns
composer_history_df_cleaned = (work_history_df[(work_history_df['work_composer_name'] != '')]
                            .set_index(['work_composer_name','work_id', 'program_id'])
                            .sort_values(['work_composer_name', 'date'])
                            .loc[:,['date','season','work_title','work_movement']]
    )

# Filter data frame for only the first time a composer is performed by the orchestra
new_composer_df = (composer_history_df_cleaned
                        .groupby('work_composer_name')
                        .first()
                        .sort_values(by='date'))

# Count the number of new composers performed in each season
new_composers_per_season = new_composer_df.season.value_counts()
new_composers_per_season.sort_index(inplace=True)

#Count of total number of composers performed per season
all_composers = work_history_df[(work_history_df['work_composer_name'] != '')]
total_composers_per_season = (all_composers[['season', 'work_composer_name']]
                        .groupby('season')
                        .nunique()
                        .rename(columns={'work_composer_name' : 'total_composers'}))

#Merge "new composer" and "total composer" counts into dataframe
composers_per_season_df = total_composers_per_season.merge(new_composers_per_season.rename('new_composers'), left_index=True, right_index=True)

# Calculate number of "repeat compsers" per season
composers_per_season_df['repeat_composers'] = (composers_per_season_df['total_composers']
                                               - composers_per_season_df['new_composers'])

# Calculate percentage of season with new/repeat composers
composers_per_season_df['new_composers_p'] = (composers_per_season_df['new_composers'] 
                                              / composers_per_season_df['total_composers'])
composers_per_season_df['repeat_composers_p'] = (composers_per_season_df['repeat_composers']
                                              / composers_per_season_df['total_composers'])

# Add season start year for filtering
composers_per_season_df['season_year'] = composers_per_season_df.index.str[:4].astype(int)

# Export to CSV
composers_per_season_df.to_csv('output/ny_phil_new_composers_per_season.csv')
